In [ ]:
import numpy as np

data = np.load("fake_embeddings.npy", allow_pickle=True).item()

db_embeddings = data["db_embeddings"]
db_labels = data["db_labels"]
db_paths = data["db_paths"]
query_embeddings = data["query_embeddings"]

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
db_proj = pca.fit_transform(db_embeddings)

plt.figure(figsize=(8, 6))
for label in np.unique(db_labels):
    idxs = db_labels == label
    plt.scatter(db_proj[idxs, 0], db_proj[idxs, 1], label=label, alpha=0.6)
plt.title("Espaço PCA dos Embeddings")
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter=1000)
scores = cross_val_score(clf, db_embeddings, db_labels, cv=5)
print(f"Acurácia média (Probing): {scores.mean():.2f}")

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

def get_top_k(query_emb, db_embs, k=5):
    dists = cosine_distances(query_emb.reshape(1, -1), db_embs).flatten()
    top_k = np.argsort(dists)[:k]
    return top_k, dists[top_k]

query_id = 0
top_ids, top_dists = get_top_k(query_embeddings[query_id], db_embeddings)
for i, idx in enumerate(top_ids):
    print(f"{i+1}. {db_paths[idx]} (dist={top_dists[i]:.4f})")

In [ ]:
import torch
import timm
import torchvision.transforms as T
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
use_vit = True  # ou False para ResNet + Grad-CAM

if use_vit:
    model = timm.create_model("vit_base_patch16_224", pretrained=True).to(device)
else:
    model = timm.create_model("resnet50", pretrained=True).to(device)

model.eval()

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3),
])

In [ ]:
def vit_attention_rollout(model, img_tensor):
    attn_blocks = [blk.attn.attn_drop for blk in model.blocks]
    img_tensor = img_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        _ = model.forward_features(img_tensor)

    attn = model.blocks[-1].attn.get_attn()
    attn = attn[0].mean(0)
    mask = attn[0, 1:].reshape(14, 14).cpu().numpy()
    return mask

In [ ]:
from torchvision.models.feature_extraction import create_feature_extractor

def gradcam_resnet(model, img_tensor):
    model.eval()
    features = {}

    def hook_fn(m, i, o): features["feat"] = o

    handle = model.layer4.register_forward_hook(hook_fn)
    img_tensor = img_tensor.unsqueeze(0).to(device)
    model(img_tensor)
    act = features["feat"].squeeze(0).mean(0).cpu().numpy()
    handle.remove()
    return act

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def visualize_explainability(img_path, use_vit=True):
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img)

    if use_vit:
        mask = vit_attention_rollout(model, img_tensor)
        mask = cv2.resize(mask, img.size)
    else:
        mask = gradcam_resnet(model, img_tensor)
        mask = cv2.resize(mask, img.size)

    mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask) + 1e-8)
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    overlay = np.array(img) * 0.5 + heatmap * 0.5

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Imagem original")
    plt.axis("off")
    plt.subplot(1, 3, 2)
    plt.imshow(heatmap)
    plt.title("Mapa de atenção")
    plt.axis("off")
    plt.subplot(1, 3, 3)
    plt.imshow(overlay.astype(np.uint8))
    plt.title("Sobreposição")
    plt.axis("off")
    plt.show()

# visualize_explainability("/caminho/para/imagem.jpg", use_vit=True)